Capturando páginas da web
==============

Neste exercício vamos deselvolver algumas técnicas de captura de páginas da web de forma eficiente, alimentando dois bancos de dados com os resultados da nossa captura: O primeiro com metadados da captura e o segundo com o conteúdo das páginas.

Lista de Links
----------------
Baixe a lista de links (https://copy.com/uT4fSbbmqNrq8B8s) e salve-a no mesmo diretório deste notebook.

**Exercício 1:** Determine o número de links a serem capturados

 


In [2]:
import pandas as pd
import gzip
import json

In [3]:
links = pd.read_csv('feeds.csv')
links

,title,link
0,Bonde. O seu portal,http://www.bonde.com.br/
1,UOL Dieta e Boa Forma,http://boaforma.uol.com.br/ultnot/
2,Migalhas,http://www.migalhas.com.br
3,Car And Driver,http://caranddriverbrasil.uol.com.br//
4,Jornal dos Concursos,http://www.jcconcursos.com.br
5,Experiências Discovery em discoverybrasil.uol....,http://discoverybrasil.uol.com.br/experiencia
6,UOL Política,http://noticias.uol.com.br/politica/
7,UOL Tabloide,http://noticias.uol.com.br/tabloide/
8,UOL Ciência,http://noticias.uol.com.br/ciencia/
9,UOL Viagem,http://viagem.uol.com.br/ultnot/


In [4]:
len(links)

264884

In [5]:
import time
from functools import wraps

def timefn(fn):
    @wraps(fn)
    def measure_time(*args, **kwargs):
        t1 = time.time()
        result = fn(*args, **kwargs)
        t2 = time.time()
        print (
            "@timefn:" + fn.__name__ + " took " + str(t2 - t1) + " seconds")
        return result
    return measure_time

**Exercício 2:** Implemente uma solução para visita sequencial às páginas correspodentes aos links.

In [13]:
import requests

In [14]:
%time
r = requests.get(links.link[0])
t = r.elapsed
t.microseconds

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 8.82 µs


792540

**Exercício 3:** Estenda o código do seu capturador para armazenar os seguintes metadados da captura em um banco SQLite: Link, status da requisição, tamanho do html em bytes.

In [10]:
import sqlite3
conn = sqlite3.connect('meubanco.sqlite', check_same_thread = False)
# O argumento Check same thread é necessário para permitir cursores criados nas threads poderem escrever no banco

In [10]:
def tabela_existe(nome, cursor):
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    nomes = cursor.fetchall()
    if len(nomes) == 0:
        return False
    #print(nomes)
    return nome in nomes[0]

cursor=conn.cursor()

cursor.execute('drop table if exists metadado')
cursor.execute('create table if not exists metadado (link text, status integer, tempo real)')

@timefn
def insere_metadado(r):
    cursor=conn.cursor()
    cursor.execute('insert into metadado values(?, ?, ?)', (r.url, r.status_code, r.elapsed.microseconds/1e6))

for link in links[:5].link:
    r = requests.get(link, verify=False)
    insere_metadado(r)
conn.commit()

@timefn:insere_metadado took 0.0003044605255126953 seconds
@timefn:insere_metadado took 3.719329833984375e-05 seconds
@timefn:insere_metadado took 5.650520324707031e-05 seconds
@timefn:insere_metadado took 5.0067901611328125e-05 seconds
@timefn:insere_metadado took 3.719329833984375e-05 seconds


In [29]:
conn = sqlite3.connect('meubanco.sqlite', check_same_thread = False)
cursor=conn.cursor()
cursor.execute('select count(*) from metadado limit 100')
resposta = cursor.fetchall()
resposta

[(88,)]

In [25]:
cursor.execute('drop table if exists metadado')
cursor.execute('create table if not exists metadado (link text, status integer, tempo real)')

**Exercício 4:** Implemente o salvamento das páginas capturadas em um banco SQLite, usando o Link como chave primária

In [50]:
conn.

**Exercício 5:** Modifique a sua implementação de captura, para que esta seja feita em paralelo. *a)* Usando Threads, *b)* Usando multiplos processos, *c)* Usando a biblioteca gevent.

In [6]:
from threading import Thread
import multiprocessing as MP
import queue
import os

In [ ]:
respostas = []
def captura(link):
    '''faz o get'''
    r = requests.get(link, verify=False)
    insere_metadado(r)
    #respostas.append(r)

for l in links[:5].link:
    print(l)
    t= Thread(target=captura, args=(l,))
    t.start()
while len(respostas)<5:
    pass
#[insere_metadado(r) for r in respostas]
conn.commit()

In [15]:
[insere_metadado(r) for r in respostas]

[None, None, None, None, None]

In [19]:
class Worker(Thread):
    def __init__(self, fila, conn, nome):
        Thread.__init__(self,name=str(nome))
        self.fila = fila
        self.conn = sqlite3.connect('meubanco.sqlite', check_same_thread = False)
        self.cursor = self.conn.cursor()
    
    def run(self):
        while True:
            try:
                # gets the link from the queue
                link = self.fila.get()

                # download the link
                print("* Thread " + self.name + " - processing URL")
                self.download_page(link)

                # send a signal to the queue that the job is done
                self.fila.task_done()
            except Exception as e:
                print(e)
                break
            finally:
                self.conn.commit()
  
    def download_page(self, link):
        r = requests.get(link)
        if (r.status_code == requests.codes.ok):
            print("* Thread: {} Downloaded {} in {} seconds".format(self.name, link, r.elapsed.microseconds/1e6))
        else:
            print("* Thread: {} failed {}: {}".format(self.name, link, r.status_code))
        
        self.cursor.execute('insert into metadado values(?, ?, ?)', (r.url, r.status_code, r.elapsed.microseconds/1e6))

class DownloadManager():
  
    def __init__(self, links, threads=5):
        self.threads = threads
        self.links = links
        self.conn = None #sqlite3.connect('meubanco.sqlite', check_same_thread = False)
        
    def pega_links(self):
        """
        Cria as threads, enche a fila de links e alimenta as threads com as urls
        """
        fila = queue.Queue()
 
        for i in range(self.threads):
            t = Worker(fila, self.conn, i)
            t.setDaemon(True)
            t.start()
        
        for link in self.links:
            #print(link)
            fila.put(link)

        # Espera a fila esvaziar
        #fila.join()
        t.conn.commit()
        return          


In [27]:
DM = DownloadManager(links.link[:100], 20)
DM.pega_links()

* Thread 0 - processing URL
* Thread 1 - processing URL
* Thread 4 - processing URL
* Thread 7 - processing URL
* Thread 16 - processing URL
* Thread 5 - processing URL
* Thread 8 - processing URL
* Thread 9 - processing URL
* Thread 10 - processing URL
* Thread 18 - processing URL
* Thread 6 - processing URL
* Thread 14 - processing URL
* Thread 15 - processing URL
* Thread 17 - processing URL
* Thread 2 - processing URL
* Thread 12 - processing URL
* Thread 13 - processing URL
* Thread 3 - processing URL
* Thread 11 - processing URL
* Thread 19 - processing URL
* Thread: 8 Downloaded http://noticias.uol.com.br/politica/ in 0.028793 seconds
* Thread: 12 failed http://tulipio.uol.com.br/cartum: 404
* Thread: 9 Downloaded http://noticias.uol.com.br/tabloide/ in 0.034718 seconds
* Thread: 13 Downloaded http://olimpiadas.uol.com.br/ultimas/ in 0.034223 seconds
* Thread: 16 Downloaded http://www.jcconcursos.com.br in 0.010579 seconds
* Thread: 17 failed http://redir.folha.com.br/redir/onli

**Exercício 6:** Faça um profiling das várias metodologias de captura.

**Exercício 7:** Análise da captura. Crie um gráfico de barras representando quantos links retornaram cada status code. 